# Extracting

Let's begin by extracting our data into pandas dataframes. I have created a simple python module just for that task. 

All dataframes will have duplicate rows deleted and empty strings will be substituted with NaN values. This is a very basic transformation that can be done for almost any data. Further transformations will depend on what we see in our dataframes.

In [90]:
from etl import extract, config
import pandas as pd

datasources = 'a'
product_invoices_path = './data/product_invoices/'
product_package_types_path = './data/product_package_types/'
product_shipments_path = './data/product_shipments/'
provider_invoices_path = './data/provider_invoices/'
provider_prices_path = './data/provider_prices/'
extractor = extract.Extractor(datasources)

product_invoices_df = extractor.load_json_files_to_df(product_invoices_path)
product_package_types_df = extractor.load_json_files_to_df(product_package_types_path)
product_shipments_df = extractor.load_json_files_to_df(product_shipments_path)
provider_invoices_df = extractor.load_json_files_to_df(provider_invoices_path)
provider_prices_df = extractor.load_json_files_to_df(provider_prices_path)


# Checking and Transforming

Now let us check our dataframes one by one and fix anything funky that might be going on with them.

I will focus on missing data and potential primary keys that are not unique.

In [44]:
# the list of transactions where we have collected the shipment fee from the buyers
product_invoices_df

,transaction_id,amount,user_invoice_date
0,221271390,3.150000,2018-12-20
1,153588634,3.150000,2018-08-24
2,270671460,2.400000,2019-03-01
3,158409145,2.730000,2018-09-04
4,178911135,2.730000,2018-10-18
...,...,...,...
55409,131206431,2.400000,2018-06-13
55410,252787333,2.400000,2019-02-08
55411,250082599,2.400000,2019-01-30
55412,244563332,2.730000,2019-01-31


In [62]:
isTransIdUnique = product_invoices_df["transaction_id"].is_unique 
print("transaction_id is Unique: " + str(isTransIdUnique) + '\n')

print('Number of missing values:')
print (product_shipments_df.isnull().sum())

transaction_id is Unique: True

Number of missing values:
tracking_code             0
from_country              0
to_country                0
package_type_id           0
transaction_id            0
shipping_label_created    0
buyer_id                  0
seller_id                 0
dtype: int64


In [64]:
#the list of package types we have on our platform that a seller can select.
product_package_types_df

,id,description
0,7,Ã‰paiss. max 3cm 0.25kg
1,8,0.75kg
2,9,2kg
3,10,0.5kg
4,11,1kg
...,...,...
129,275,"Max. 20 kg, 120 x 60 x 60 cm"
130,276,"Max. 20 kg, 120 x 60 x 60 cm"
131,277,"Max. 20 kg, 120 x 60 x 60 cm"
132,292,"Max. 10 kg, 100 Ã— 50 Ã— 50 cm"


In [66]:
isIdUnique = product_package_types_df["id"].is_unique 
print("id is Unique: " + str(isIdUnique) + '\n')

print('Number of missing values:')
print (product_package_types_df.isnull().sum())

id is Unique: True

Number of missing values:
id             0
description    8
dtype: int64


We see 8 missing description values. I will not drop them because I suspect their IDs might still be associated with data from the other tables. The description data also looks quite ugly. It includes both dimensions and weight of the packages. Dimentions are not referred to in any of the other tables. We will transform this data so that the max kg value is saved in a separate column. This way we can make the most of the product_package data - for example, we can compare whether the shipping information matches the information provided by the sellers.

In [67]:
#the list of shipments that we see in our data
product_shipments_df

,tracking_code,from_country,to_country,package_type_id,transaction_id,shipping_label_created,buyer_id,seller_id
0,30611711,FR,FR,11,182260673,2018-10-13 10:38:35,16471055,18098417
1,33192017,FR,FR,10,203716496,2018-11-14 18:43:26,511189,16281325
2,33286778,FR,FR,11,203394963,2018-11-15 21:39:24,13351339,15644819
3,28134416,FR,FR,11,161640430,2018-09-06 09:44:07,18319157,855863
4,29892135,FR,FR,10,174990551,2018-10-04 08:37:26,16998722,1691012
...,...,...,...,...,...,...,...,...
56291,33409354,FR,FR,12,200390728,2018-11-17 18:21:51,19699917,16662926
56292,40949324,FR,FR,10,265068264,2019-02-17 19:14:26,11879770,15749068
56293,33929048,FR,FR,10,210053751,2018-11-24 08:56:13,11880641,15285615
56294,28406245,FR,FR,10,164804384,2018-09-10 18:46:39,17863854,11502596


In [72]:
isTransactionIdUnique = product_shipments_df["transaction_id"].is_unique 
print("transaction_id is Unique: " + str(isTransactionIdUnique) + '\n')
isTrackingCdUnique = product_shipments_df["tracking_code"].is_unique 
print("tracking_code is Unique: " + str(isTrackingCdUnique) + '\n')

print('Number of missing values:')
print (product_shipments_df.isnull().sum())

transaction_id is Unique: True

tracking_code is Unique: False

Number of missing values:
tracking_code             0
from_country              0
to_country                0
package_type_id           0
transaction_id            0
shipping_label_created    0
buyer_id                  0
seller_id                 0
dtype: int64


In [91]:
ids = product_shipments_df["tracking_code"]
product_shipments_df_duplicated_tracking = product_shipments_df[ids.isin(ids[ids.duplicated()])]
product_shipments_df_duplicated_tracking

,tracking_code,from_country,to_country,package_type_id,transaction_id,shipping_label_created,buyer_id,seller_id
15069,12878564,FR,FR,11,8638716,2015-05-19 18:50:57,1702596,105141
52011,12878564,ES,FR,135,271780177,2019-02-26 12:14:31,11869825,21649195
3124,12795958,FR,FR,11,7789938,2015-04-25 20:07:56,1827490,1164460
7354,12795958,ES,FR,135,260812237,2019-02-11 16:54:05,21341323,22008389


We can see that there are 4 duplicated tracking code values. This seems like a data quality issue, which needs explaination - these codes are cleraly associated with different shipments. For now, I will drop this data so we can use the tracking_code to join different dataframes. 

In [92]:
duplicates = [12878564, 12795958]
product_shipments_df = product_shipments_df.drop(product_shipments_df[ids.isin(duplicates)].index)
isTrackingCdUnique = product_shipments_df["tracking_code"].is_unique 
print("tracking_code is Unique: " + str(isTrackingCdUnique) + '\n')

tracking_code is Unique: True



In [93]:
product_shipments_df

,tracking_code,from_country,to_country,package_type_id,transaction_id,shipping_label_created,buyer_id,seller_id
0,30611711,FR,FR,11,182260673,2018-10-13 10:38:35,16471055,18098417
1,33192017,FR,FR,10,203716496,2018-11-14 18:43:26,511189,16281325
2,33286778,FR,FR,11,203394963,2018-11-15 21:39:24,13351339,15644819
3,28134416,FR,FR,11,161640430,2018-09-06 09:44:07,18319157,855863
4,29892135,FR,FR,10,174990551,2018-10-04 08:37:26,16998722,1691012
...,...,...,...,...,...,...,...,...
56291,33409354,FR,FR,12,200390728,2018-11-17 18:21:51,19699917,16662926
56292,40949324,FR,FR,10,265068264,2019-02-17 19:14:26,11879770,15749068
56293,33929048,FR,FR,10,210053751,2018-11-24 08:56:13,11880641,15285615
56294,28406245,FR,FR,10,164804384,2018-09-10 18:46:39,17863854,11502596


In [75]:
# the shipping labels that the shipping provider is charging us for.
provider_invoices_df
isTrackingCdUnique = provider_invoices_df["tracking_code"].is_unique 
print("tracking_cd is Unique: " + str(isTrackingCdUnique) + '\n')
print (provider_invoices_df.isnull().sum())
provider_invoices_df

ids = provider_invoices_df["tracking_code"]
provider_invoices_df_duplicated_tracking = provider_invoices_df[ids.isin(ids[ids.duplicated()])]
provider_invoices_df_duplicated_tracking

tracking_cd is Unique: False

tracking_code         0
from_country          0
to_country            0
weight_measured    3333
amount                0
dtype: int64


,tracking_code,from_country,to_country,weight_measured,amount
44893,12795958,FR,FR,NaN,3.28
5704,12795958,ES,FR,0.0,3.73
29611,12878564,FR,FR,NaN,3.28
9082,12878564,ES,FR,0.0,3.73


The same repeated tracking_ids can be found in the provider_invoices data and we will get rid of them for the sake of easier exploration of trends. 

In [88]:
provider_invoices_df = provider_invoices_df.drop(provider_invoices_df[ids.isin(duplicates)].index)
isTrackingCdUnique = provider_invoices_df["tracking_code"].is_unique 
print("tracking_code is Unique: " + str(isTrackingCdUnique) + '\n')

tracking_code is Unique: True



I wonder whether the repeated tracking code will be found in the product_invoices data - we can find that based on the uniqie transaction ids we saw for the tracking codes in the  product_shipments data.

In [63]:
product_invoices_df

duplicates = [8638716, 271780177, 7789938, 260812237]

product_invoices_df.loc[product_invoices_df['transaction_id'].isin(duplicates)]
# product_invoices_df = product_invoices_df.set_index(['transaction_id'])
# print(product_invoices_df.loc[7789938])

,transaction_id,amount,user_invoice_date
44796,8638716,2.730000,2015-05-26
12882,271780177,3.727273,2019-03-06
40042,7789938,2.730000,2015-05-05
14834,260812237,3.727273,2019-02-19


Yup, it is here. We can drop it but when we join this table with the others, it won't matter as the values will be dropped anyway.

In [6]:
#the prices that we are being charged by the provider based on weight, route of the shipment.
provider_prices_df

,from_country,to_country,price,actual_package_size
0,FR,FR,2.40,0.5 kg
1,FR,FR,2.73,1 kg
2,FR,FR,3.15,2 kg
3,FR,FR,3.54,3 kg
4,FR,FR,4.27,5 kg
...,...,...,...,...
139,ES,LU,6.77,5 kg
140,ES,LU,8.96,7 kg
141,ES,LU,11.75,10 kg
142,ES,LU,15.58,15 kg


This seems like a reference data from the provider, which we can use to double-check whether the provider charges us fairly.

To make things easier - let's join product_shipments_df, product_invoices_df and provider_invoices_df! This will allow us to have one main dataset, which might make explorations easier. 

In [130]:
import numpy as np
product_invoices_shipments_df = pd.merge(product_shipments_df, product_invoices_df, on='transaction_id')
# product_invoices_shipments_df.rename(columns={'from_country':'from_country_product', 'to_country':'to_country_product'}, inplace = True)
# product_invoices_shipments_df
product_provider_invoices_shipments_df = pd.merge(product_invoices_shipments_df, provider_invoices_df, on=['tracking_code','tracking_code'])
product_provider_invoices_shipments_df

# comparison_from_country = np.where(product_provider_invoices_shipments_df["from_country_x"] == product_provider_invoices_shipments_df["from_country_y"], True, False)

# product_provider_invoices_shipments_df

# comparing columns
# isAmtSame = product_provider_invoices_shipments_df['amount_x'].equals(product_provider_invoices_shipments_df['amount_y'])
# print(isAmtSame)

,tracking_code,from_country_x,to_country_x,package_type_id,transaction_id,shipping_label_created,buyer_id,seller_id,amount_x,user_invoice_date,from_country_y,to_country_y,weight_measured,amount_y
0,30611711,FR,FR,11,182260673,2018-10-13 10:38:35,16471055,18098417,2.73,2018-10-22,FR,FR,960.0,2.73
1,33192017,FR,FR,10,203716496,2018-11-14 18:43:26,511189,16281325,2.40,2018-11-24,FR,FR,240.0,2.40
2,33286778,FR,FR,11,203394963,2018-11-15 21:39:24,13351339,15644819,2.73,2018-12-02,FR,FR,0.0,2.73
3,28134416,FR,FR,11,161640430,2018-09-06 09:44:07,18319157,855863,2.73,2018-09-17,FR,FR,970.0,2.73
4,29892135,FR,FR,10,174990551,2018-10-04 08:37:26,16998722,1691012,2.40,2018-10-07,FR,FR,200.0,2.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553995,33409354,FR,FR,12,200390728,2018-11-17 18:21:51,19699917,16662926,3.15,2018-11-25,FR,FR,1260.0,3.15
553996,40949324,FR,FR,10,265068264,2019-02-17 19:14:26,11879770,15749068,2.40,2019-02-22,FR,FR,0.0,2.40
553997,33929048,FR,FR,10,210053751,2018-11-24 08:56:13,11880641,15285615,2.40,2018-11-30,FR,FR,0.0,2.40
553998,28406245,FR,FR,10,164804384,2018-09-10 18:46:39,17863854,11502596,2.40,2018-09-15,FR,FR,160.0,2.40


We have dropped some records when joining product_shipments_df and product_invoices_df

# Exploring

In [131]:
product_provider_invoices_shipments_df.describe()

#Q1when is the discrepancy observed - country, buyer, seller, timing, weight???

,tracking_code,package_type_id,transaction_id,buyer_id,seller_id,amount_x,weight_measured,amount_y
count,5.540000e+05,554000.000000,5.540000e+05,5.540000e+05,5.540000e+05,554000.000000,550848.000000,554000.000000
mean,3.007754e+07,11.118292,1.750845e+08,1.326971e+07,1.323260e+07,2.654766,455.145214,2.644442
std,8.814713e+06,7.145920,7.359586e+07,5.270592e+06,5.502852e+06,0.289454,542.064702,0.340497
min,1.251388e+07,10.000000,8.542080e+05,1.830000e+02,9.800000e+01,2.380000,0.000000,2.400000
25%,2.255762e+07,10.000000,1.151012e+08,1.215802e+07,1.215069e+07,2.400000,0.000000,2.400000
50%,3.017772e+07,11.000000,1.782365e+08,1.400599e+07,1.407971e+07,2.730000,320.000000,2.400000
75%,3.770648e+07,11.000000,2.383268e+08,1.652077e+07,1.669657e+07,2.730000,670.000000,2.730000
max,4.535205e+07,153.000000,3.000001e+08,2.355383e+07,2.354916e+07,11.866667,26880.000000,15.580000


Q2Growth rate: Do number of orders increase with time? Do number of sellers increase with time? how many shippings does a seller do? how many  orders does a single buyer receive? are packages becoming heavier? trends according to countries

Churn - the rate of losing customers (retention rate)

Q3 if numbers of shippings are increasing, Vinted could negotiate lower prices with the shipping companies as it is becoming a bigger customer